# Life Expectancy vs Birth Rate by Country Between 1950 - 2015

In [1]:
### Life expectancy, birth rate (fertility) and population files were merged together and
### data was cleaned before creating the visuals.

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('../data/lifexp_birth_merged.csv')

In [3]:
# Delete the 'Unnamed: 0' column
column_to_delete = 'Unnamed: 0'
df.drop(columns=column_to_delete, inplace=True)

In [4]:
# Create a subset of data for one year
year_filter = df['year'] == 2015
df_subset = df[year_filter]
df_subset.head()

,country,year,life expectancy,continent,population,fertility
65,Afghanistan,2015,53.8,Asia,32526562.0,4.47
131,Albania,2015,78.0,Europe,2896679.0,1.78
197,Algeria,2015,76.4,Africa,39666519.0,2.71
263,Angola,2015,59.6,Africa,25021974.0,5.65
329,Antigua and Barbuda,2015,76.4,North America,91818.0,2.06


In [9]:
# Change scientific notation and add separator to large numbers
pd.set_option("display.float_format", lambda x: '%.2f' % x)
pd.options.display.float_format = '{:,.0f}'.format
df_subset.describe()

,year,life expectancy,population,fertility
count,168,168,168,168
mean,"2,015",72,"41,372,235",3
std,0,8,"150,577,788",1
min,"2,015",47,"91,818",1
25%,"2,015",66,"2,857,138",2
50%,"2,015",74,"9,624,897",2
75%,"2,015",78,"28,858,647",4
max,"2,015",83,"1,376,048,943",8


In [58]:
filt = df_subset['country'] == 'Nigeria'
df_subset[filt]

,country,year,life expectancy,continent,population,fertility
7457,Nigeria,2015,65,Africa,"182,201,962",6


In [ ]:
# Dynamic visualization of data

# For loop over the years
for year in range(1950, 2016):
    df_subset = df[df['year'] == year]
    
    plt.figure(figsize=(20, 12))
    plt.title(f'Life Expectancy vs Birth Rate by Countries ({year})', fontsize=22)
    plt.ylabel('Birth Rate', fontsize=20)
    plt.xlabel('Life Expectancy', fontsize=20)

    # Use a scatter plot and set the specifications of the plot
    size_transformed = np.sqrt(df_subset['population'])
    sns.scatterplot(x='life expectancy', y='fertility', hue='continent', 
                    size=size_transformed, data=df_subset, alpha=0.9, sizes=(50, 5000), 
                    palette={'Asia': "#82dfab", 'Europe': "#1b85b8", 'Africa': "#ffb77e",
                             'North America': "#AAAAAA", 'South America': "#faeca0",
                             'Australia and Oceania': "#ec6064"})  

    # Set the background color using matplotlib
    plt.gca().set_facecolor('#e0e0e0') 

    # Get handles and labels from the legend
    handles, labels = plt.gca().get_legend_handles_labels()
    population_scaling_factor = 1e-5

    # Filter out handles and labels for population sizes
    n_continents = len(df_subset['continent'].unique())
    handles, labels = handles[:n_continents + 1], labels[:n_continents + 1]

    # Position the modified legend in the top right corner
    plt.legend(handles, labels, loc='upper right', fontsize=17, markerscale=3, frameon=False)

    # Set the range for x and y axis
    plt.axis([10, 85, 0, 9])

    # Define a function to annotate some countries
    def annotate_country(country_name, df_subset):
        country_data = df_subset[df_subset['country'] == country_name]
        plt.annotate(country_name, (country_data['life expectancy'], country_data['fertility']),
                     textcoords="offset points", xytext=(10, 10), ha='center', fontsize=12, color='black')

    # Annotate the countries and use it with a for loop to iterate
    countries_to_annotate = ['China', 'India', 'United States', 'Germany',
                              'Nigeria', 'Lesotho', 'Niger',
                             'Brazil', 'South Africa', 'Mexico', 'Egypt']

    for country in countries_to_annotate:
        annotate_country(country, df_subset)

    # Save the plot for each year
    plt.savefig(f'life_expectancy_vs_birth_rate_{year}.png')

    # Close the plot
    plt.close()


In [ ]:
# Generate a GIF file from the individual pictures with for loop
import imageio

images = []

for year in range(1950, 2016):
    filename = f'life_expectancy_vs_birth_rate_{year}.png'
    images.append(imageio.imread(filename))

imageio.mimsave(f'all_years_combined.gif', images, fps=8)